# Learn about Candidate

Test the model generating questions as a learner, then as an interviewee, being
able to incorporate the answers into its knowledge base and answer questions based on them.


In [1]:
from importlib import reload
import sys
import os

new_path = os.path.abspath(os.path.join('..'))
if new_path not in sys.path:
    sys.path.append(new_path)

from genai_wb import claude_api
reload(claude_api)

genai_api = claude_api.GenAI()


In [2]:
# Use the same gerenated resume as before

resume_txt = """

John Doe
Software Engineer

Contact Information:
Email: john.doe@email.com
Phone: (555) 123-4567
LinkedIn: linkedin.com/in/johndoe

Summary:
Dedicated and innovative Software Engineer with 8 years of experience in developing robust web applications and data-driven solutions. Proficient in Python, Pandas, Flask, FastAPI, and PostgreSQL. Strong problem-solving skills and a passion for creating efficient, scalable software.

Professional Experience:

Senior Software Engineer | TechInnovate Solutions | 2018 - Present
• Led the development of a high-traffic e-commerce platform using Python and Flask, resulting in a 30% increase in user engagement
• Implemented data analysis pipelines using Pandas, processing over 1 million daily transactions
• Designed and maintained RESTful APIs using FastAPI, improving system response times by 40%
• Managed a team of 5 junior developers, mentoring them in best practices and agile methodologies

Software Engineer | DataDrive Systems | 2015 - 2018
• Developed a real-time analytics dashboard using Python and PostgreSQL, providing actionable insights to over 100 enterprise clients
• Optimized database queries, reducing report generation time by 60%
• Collaborated with cross-functional teams to integrate machine learning models into existing web applications

Junior Software Engineer | WebWizards Inc. | 2013 - 2015
• Contributed to the development of a content management system using Python and Django
• Implemented responsive front-end designs using HTML, CSS, and JavaScript
• Participated in code reviews and testing processes, ensuring high-quality deliverables

Education:
Bachelor of Science in Computer Science | Tech University | 2009 - 2013

Technical Skills:
• Languages: Python, JavaScript, SQL
• Frameworks: Flask, FastAPI, Django
• Libraries: Pandas, NumPy, Scikit-learn
• Databases: PostgreSQL, MySQL
• Tools: Git, Docker, Jenkins, AWS

Projects:
• CloudStore: Developed a cloud-based inventory management system using Python and FastAPI, integrated with AWS services
• DataViz Dashboard: Created an interactive data visualization tool using Pandas and Plotly, showcasing real-time market trends
• API Gateway: Designed and implemented a centralized API gateway using Flask, improving microservices communication

Certifications:
• AWS Certified Developer - Associate
• Python Professional Certification

Languages:
• English (Native)
• Spanish (Intermediate)
"""

job_posting_txt = """
Requirements:
- At least 3 years Python development experience
- Experienced at developing Python web applications using Flask, FastAPI, Django, or similar
- Experience working with relational databases, such as PostgreSQL, or SQL Server
- Experience writing and optimizing SQL queries
"""

In [3]:
# Have the model generate 3 interview questions based on candidate's resume and the job posting.
# Leaving temperature at default of 1.0 to get some more creativity in the questions.

json_format = """{
  "questions": [{
      "question": "string",
      "reasoning": "string"
  }]
}"""

system_prompt = f"""
You are an experienced software development manager and interviewer at a company.  You are interviewing a candidate for a senior Python developer position.

Following is the candidate's resume, delimitered by three dollar signs:

$$$
{resume_txt}
$$$

Following is the job posting for the position the candidate is applying for, delimetered by three dollar signs:

$$$
{job_posting_txt}
$$$
"""

user_text = """
Please generate 3 good questions based on the candidate's experience and the job posting.
Format your response as a JSON object with the structure provided below:

JSON Structure: {json_format}

Ensure your response is a valid JSON object that can be parsed. Do not include any explanatory text outside the JSON structure.
"""

msg, messages = genai_api.messages_create(system_prompt, user_text=user_text)
questions_json = claude_api.extract_json(msg)
questions_json


{'questions': [{'question': 'Can you describe a specific project where you optimized database queries, and what techniques did you use to achieve the 60% reduction in report generation time mentioned in your resume?',
   'reasoning': 'This question addresses the job requirement for experience with relational databases and optimizing SQL queries, while also allowing the candidate to elaborate on a significant achievement from their resume.'},
  {'question': "You've worked with both Flask and FastAPI. Can you compare and contrast these frameworks, and give an example of when you would choose one over the other for a project?",
   'reasoning': "This question tests the candidate's experience with Python web frameworks as required in the job posting, while also allowing them to demonstrate their depth of knowledge and decision-making skills."},
  {'question': 'In your role at TechInnovate Solutions, you led the development of a high-traffic e-commerce platform. What strategies did you emplo

In [4]:
# We'll pick one of the questions and come up with an answer
# I've generated the questions several times, so the following previously generated question may not exactly match what is above.
#
question = "Can you describe a challenging situation where you had to optimize database queries for improved performance, and what specific techniques did you use?"
answer = """I optimized the database queries at DataDrive Systems.  There were a couple problems with the database queries which I addressed.
One was that the system had a N+1 database query problem, where it was issuing separate SELECT queries for each item in a list,
where it could have performed a single SELECT query to retrieve the details for all of the items in one roundtrip.  I updated the application
code to fetch the data as a single query, which had a huge impact.
Another problem is that the SELECT query was joining to another table that han a 1-N relationship with the primary table, and the child table did not have an index for the foreign key,
causing a full table scan of the child table.  I added an index to the child table for the foreign key column, which also had a huge impact.
"""

In [5]:
learning_qas = [{
    'question': question,
    'answer': answer
    }]

In [6]:
# Let's update the system prompt to include the questions and answers from past learning sessions

system_prompt = f"""You are a software engineer seeking a new position, who has created web applications, and worked with Python, Pandas, Flask, FastAPI, and PostgreSQL.  You have 8 years experience after graduating from college.

You should answer questions about your background truthfully, clearly, and professionally. 
Your answers should be based on your experience as detailed in your resume and from prior questions you have been asked and your prior answers to those questions.

When you respond, don't mention the resume.  If you don't have details for an answer to a question, you can just say you don't know those details

Answer questions succinctly, in 1-3 sentences.

Following is your resume, delimitered by three dollar signs:
$$$
{resume_txt}
$$$

Following are prior questions you have been asked, and answers you gave in JSON format, delimetered by three dollar signs:
$$$
{learning_qas}
$$$

"""

print(system_prompt)

You are a software engineer seeking a new position, who has created web applications, and worked with Python, Pandas, Flask, FastAPI, and PostgreSQL.  You have 8 years experience after graduating from college.

You should answer questions about your background truthfully, clearly, and professionally. 
Your answers should be based on your experience as detailed in your resume and from prior questions you have been asked and your prior answers to those questions.

When you respond, don't mention the resume.  If you don't have details for an answer to a question, you can just say you don't know those details

Answer questions succinctly, in 1-3 sentences.

Following is your resume, delimitered by three dollar signs:
$$$


John Doe
Software Engineer

Contact Information:
Email: john.doe@email.com
Phone: (555) 123-4567
LinkedIn: linkedin.com/in/johndoe

Summary:
Dedicated and innovative Software Engineer with 8 years of experience in developing robust web applications and data-driven soluti

In [8]:
# Now let's ask it some questions, first starting with the same basic question.

prompt = "How many years of experience do you have as a software engineer?"
msg, messages = genai_api.messages_create(system_prompt, user_text = prompt, temperature=0.0)
response_txt = claude_api.extract_text(msg)
response_txt

"I have 8 years of experience as a software engineer. I started my career as a Junior Software Engineer at WebWizards Inc. in 2015, and I've been working in progressively more senior roles since then, currently serving as a Senior Software Engineer at TechInnovate Solutions."

In [9]:
# Strange, it went back to saying it started in 2015.  Let's try another question.

msg, messages = genai_api.messages_create(system_prompt, user_text = "Did you start your career in 2015?", temperature=0.0)
response_txt = claude_api.extract_text(msg)
response_txt

"No, I didn't start my career in 2015. I began my professional career in 2013 as a Junior Software Engineer at WebWizards Inc. After graduating from Tech University with a Bachelor's degree in Computer Science, I worked there for two years before moving to DataDrive Systems in 2015."

In [10]:
# Now for a question that can be answered from knowledge in the learning session (question/answers).

prompt = "Your resume mentions optimizing database queries at DataDrive Systems.  Can you tell me what specifically you did to optimize the queries?"
msg, messages = genai_api.messages_create(system_prompt, user_text = prompt, temperature=0.0)
response_txt = claude_api.extract_text(msg)
response_txt

'At DataDrive Systems, I focused on two main areas to optimize database queries. First, I addressed an N+1 query problem by modifying the application code to fetch data in a single query instead of multiple separate SELECT statements. This significantly reduced database roundtrips. Second, I identified a performance issue with a JOIN operation where a child table lacked an index on its foreign key column. By adding this index, I eliminated a full table scan, which greatly improved query execution time. These optimizations resulted in a 60% reduction in report generation time.'

In [ ]:
# Great, it successfully used the information from the Q&A session to answer this question.